## Create datasets for the Content-based Filter

This notebook builds the data we will use for creating our content based model. We'll collect the data via a collection of SQL queries from the publicly avialable Kurier.at dataset in BigQuery.
Kurier.at is an Austrian newsite. The goal of these labs is to recommend an article for a visitor to the site. In this lab we collect the data for training. In the subsequent notebook we train the recommender model. 

This notebook illustrates
* how to pull data from BigQuery table and write to local files
* how to make reproducible train and test splits 

In [1]:
import os
import tensorflow as tf
import numpy as np
import google.datalab.bigquery as bq

PROJECT = 'qwiklabs-gcp-2737edcaba61fe20' # REPLACE WITH YOUR PROJECT ID
BUCKET = 'qwiklabs-gcp-2737edcaba61fe20' # REPLACE WITH YOUR BUCKET NAME
REGION = 'us-central1' # REPLACE WITH YOUR BUCKET REGION e.g. us-central1

# do not change these
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '1.8'

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
%%bash
gcloud  config  set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


We will use this helper funciton to write lists containing article ids, categories, and authors for each article in our database to local file. 

In [3]:
def write_list_to_disk(my_list, filename):
  with open(filename, 'w') as f:
    for item in my_list:
        line = "%s\n" % item
        f.write(str(line.encode('utf8')))

### Pull data from BigQuery

The cell below creates a local text file containing all the article ids (i.e. 'content ids') in the dataset. 

Have a look at the original dataset in [BigQuery](https://bigquery.cloud.google.com/welcome/). Then read through the query below and make sure you understand what it is doing. 

In [4]:
sql="""
#standardSQL
SELECT  
  (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) AS content_id 
FROM `cloud-training-demos.GA360_test.ga_sessions_sample`,   
  UNNEST(hits) AS hits
WHERE 
  # only include hits on pages
  hits.type = "PAGE"
  AND (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) IS NOT NULL
GROUP BY
  content_id
"""
content_ids_list = bq.Query(sql).execute().result().to_dataframe()['content_id'].tolist()
write_list_to_disk(content_ids_list, "content_ids.txt")
print("Some sample content IDs {}".format(content_ids_list[:3]))
print("The total number of articles is {}".format(len(content_ids_list)))

Some sample content IDs ['299837992', '254619647', '299972800']
The total number of articles is 15634


In [5]:
# try and figure out the different hit types
# also check out this whole 'unnest' thing they've got going on with the data structure...
# not sure how they've wierded up SQL
sql="""
SELECT  distinct
  hits.type as hit_type # figure out hit types 
FROM `cloud-training-demos.GA360_test.ga_sessions_sample`,   
  UNNEST(hits) AS hits
"""
hit_types = bq.Query(sql).execute().result().to_dataframe()['hit_type'].tolist()
print(hit_types)

['EVENT', 'PAGE', 'SOCIAL']


In the following cells, you will create a local file which contains a list of article categories and a list of article authors.

**Hint**: For the TODO below, modify the query above changing 'content_id' to the necessary field and changing index=10 
to the appropriate index.  
Refer back to the original dataset, use the `hits.customDimensions.index` 
field to find the correct index.

In [8]:
#TODO: Modify the above query to instead create a list of all categories in the dataset.
#You'll need to change the content_id to the appropriate field as well as the index. 
sql="""
#standardSQL
SELECT  
  (SELECT MAX(IF(index=7, value, NULL)) FROM UNNEST(hits.customDimensions)) AS category  
FROM `cloud-training-demos.GA360_test.ga_sessions_sample`,   
  UNNEST(hits) AS hits
WHERE 
  # only include hits on pages
  hits.type = "PAGE"
  AND (SELECT MAX(IF(index=7, value, NULL)) FROM UNNEST(hits.customDimensions)) IS NOT NULL
GROUP BY   
  category
"""
categories_list = bq.Query(sql).execute().result().to_dataframe()['category'].tolist()
#TODO: Modify the query above to create the list of categories
write_list_to_disk(categories_list, "categories.txt")
print(categories_list)

['Stars & Kultur', 'News', 'Lifestyle']


When creating the author list, we'll only use the first author information for each article.  

In [6]:
sql="""
#standardSQL
SELECT
  REGEXP_EXTRACT((SELECT MAX(IF(index=2, value, NULL)) FROM UNNEST(hits.customDimensions)), r"^[^,]+")  AS first_author  
FROM `cloud-training-demos.GA360_test.ga_sessions_sample`,   
  UNNEST(hits) AS hits
WHERE 
  # only include hits on pages
  hits.type = "PAGE"
  AND (SELECT MAX(IF(index=2, value, NULL)) FROM UNNEST(hits.customDimensions)) IS NOT NULL
GROUP BY   
  first_author
"""
authors_list = bq.Query(sql).execute().result().to_dataframe()['first_author'].tolist()
write_list_to_disk(authors_list, "authors.txt")
print("Some sample authors {}".format(authors_list[:10]))
print("The total number of authors is {}".format(len(authors_list)))

Some sample authors ['Christina Michlits', 'Mathias Kainz', 'Thomas  Trescher', 'Stefan Berndl', 'Anita Kattinger', 'Martina Salomon', 'Marlene Patsalidis', 'Georg Leyrer', 'Elisabeth Spitzer', 'Elisabeth Sereda']
The total number of authors is 385


### Create train and test sets.

In this section, we will create the train/test split of our data for training our model. Read through the query and complete the TODO at the bottom.  
Use the concatenated values for visitor id and content id to create a farm fingerprint, taking 90% of the data for the training set. 

In [11]:
sql="""
WITH site_history as (
  SELECT
      fullVisitorId as visitor_id,
      (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) AS content_id,
      (SELECT MAX(IF(index=7, value, NULL)) FROM UNNEST(hits.customDimensions)) AS category, 
      (SELECT MAX(IF(index=6, value, NULL)) FROM UNNEST(hits.customDimensions)) AS title,
      (SELECT MAX(IF(index=2, value, NULL)) FROM UNNEST(hits.customDimensions)) AS author_list,
      SPLIT(RPAD((SELECT MAX(IF(index=4, value, NULL)) FROM UNNEST(hits.customDimensions)), 7), '.') as year_month_array,
      LEAD(hits.customDimensions, 1) OVER (PARTITION BY fullVisitorId ORDER BY hits.time ASC) as nextCustomDimensions
  FROM 
    `cloud-training-demos.GA360_test.ga_sessions_sample`,   
     UNNEST(hits) AS hits
   WHERE 
     # only include hits on pages
      hits.type = "PAGE"
      AND
      fullVisitorId IS NOT NULL
      AND
      hits.time != 0
      AND
      hits.time IS NOT NULL
      AND
      (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) IS NOT NULL
)
SELECT
  visitor_id,
  content_id,
  category,
  REGEXP_REPLACE(title, r",", "") as title,
  REGEXP_EXTRACT(author_list, r"^[^,]+") as author,
  DATE_DIFF(DATE(CAST(year_month_array[OFFSET(0)] AS INT64), CAST(year_month_array[OFFSET(1)] AS INT64), 1), DATE(1970,1,1), MONTH) as months_since_epoch,
  (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(nextCustomDimensions)) as next_content_id
  --, ABS(FARM_FINGERPRINT(CONCAT(visitor_id, content_id))) as farm_id -- don't actually need this in the file
FROM
  site_history
WHERE (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(nextCustomDimensions)) IS NOT NULL 
  AND MOD(ABS(FARM_FINGERPRINT(CONCAT(visitor_id, content_id))), 10) < 9 -- remember index starts at 0
  --AND RAND() < 0.0005 -- if random sampling further required
"""
# TODO: Use FARM_FINGERPRINT on the concatenated visitor_id and content_id to create a training set of approximately 90% of the data
# the hash output can't be used in the same pass
training_set_df = bq.Query(sql).execute().result().to_dataframe()
training_set_df.to_csv('training_set.csv', header=False, index=False, encoding='utf-8')
training_set_df.head()


,visitor_id,content_id,category,title,author,months_since_epoch,next_content_id
0,1000196974485173657,299836841,News,ÖVP will Studiengebühren FPÖ in Verhandlungen ...,Raffaela Lindorfer,574,299959410
1,1000196974485173657,299959410,News,Koalition: Bildungspapier mit mehr Pflichten u...,Peter Temel,574,299925086
2,1000196974485173657,299925086,News,Marihuana-Adventkalender findet in Kanada reiß...,None,574,299826775
3,1000196974485173657,299826775,Lifestyle,Auf Bank ausgeruht: Pensionist muss Strafe zahlen,Marlene Patsalidis,574,299930679
4,1000196974485173657,299930679,News,Wintereinbruch naht: Erster Schnee im Osten mö...,Daniela Wahl,574,299950903


In [13]:
# full size is 257907,8
training_set_df.shape  # 90% should give ~232308

(232308, 7)

Repeat the query as above but change outcome of the farm fingerprint hash to collect the remaining 10% of the data for the test set.

In [14]:
sql="""
WITH site_history as (
  SELECT
      fullVisitorId as visitor_id,
      (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) AS content_id,
      (SELECT MAX(IF(index=7, value, NULL)) FROM UNNEST(hits.customDimensions)) AS category, 
      (SELECT MAX(IF(index=6, value, NULL)) FROM UNNEST(hits.customDimensions)) AS title,
      (SELECT MAX(IF(index=2, value, NULL)) FROM UNNEST(hits.customDimensions)) AS author_list,
      SPLIT(RPAD((SELECT MAX(IF(index=4, value, NULL)) FROM UNNEST(hits.customDimensions)), 7), '.') as year_month_array,
      LEAD(hits.customDimensions, 1) OVER (PARTITION BY fullVisitorId ORDER BY hits.time ASC) as nextCustomDimensions
  FROM 
    `cloud-training-demos.GA360_test.ga_sessions_sample`,   
     UNNEST(hits) AS hits
   WHERE 
     # only include hits on pages
      hits.type = "PAGE"
      AND
      fullVisitorId IS NOT NULL
      AND
      hits.time != 0
      AND
      hits.time IS NOT NULL
      AND
      (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) IS NOT NULL
)
SELECT
  visitor_id,
  content_id,
  category,
  REGEXP_REPLACE(title, r",", "") as title,
  REGEXP_EXTRACT(author_list, r"^[^,]+") as author,
  DATE_DIFF(DATE(CAST(year_month_array[OFFSET(0)] AS INT64), CAST(year_month_array[OFFSET(1)] AS INT64), 1), DATE(1970,1,1), MONTH) as months_since_epoch,
  (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(nextCustomDimensions)) as next_content_id
FROM
  site_history
WHERE (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(nextCustomDimensions)) IS NOT NULL
--TODO: Modify the FARM_FINGERPRINT you used in the previous cell to create a test set of approximately 10% of the data
  AND MOD(ABS(FARM_FINGERPRINT(CONCAT(visitor_id, content_id))), 10) = 9 -- remember index starts at 0

"""

test_set_df = bq.Query(sql).execute().result().to_dataframe()
test_set_df.to_csv('test_set.csv', header=False, index=False, encoding='utf-8')
test_set_df.head()

,visitor_id,content_id,category,title,author,months_since_epoch,next_content_id
0,1000025265994336570,299816215,News,Fahnenskandal von Mailand: Die Austria zeigt F...,Alexander Strecha,574,299954138
1,1000163602560555666,299814775,Lifestyle,Meghan Markle: Verlobungsring aus Dianas Brosche,Maria Zelenko,574,299907204
2,1000163602560555666,299972800,News,Strengere Regeln für Schüler Eltern – und auch...,None,574,299824032
3,1000163602560555666,299972800,News,Strengere Regeln für Schüler Eltern – und auch...,None,574,299809748
4,1000295479560700003,299836255,News,Blümel Kneissl &Co.: Das sind die Fixstarter,None,574,299833840


In [15]:
test_set_df.shape

(25599, 7)

Let's have a look at the two csv files we just created containing the training and test set. We'll also do a line count of both files to confirm that we have achieved an approximate 90/10 train/test split.  
In the next notebook, **Content Based Filtering** we will build a model to recommend an article given information about the current article being read, such as the category, title, author, and publish date. 

In [16]:
%%bash
wc -l *_set.csv

   25599 test_set.csv
  232308 training_set.csv
  257907 total


In [17]:
!head *_set.csv

==> test_set.csv <==
1000025265994336570,299816215,News,Fahnenskandal von Mailand: Die Austria zeigt Flagge,Alexander Strecha,574,299954138
1000163602560555666,299814775,Lifestyle,Meghan Markle: Verlobungsring aus Dianas Brosche,Maria Zelenko,574,299907204
1000163602560555666,299972800,News,Strengere Regeln für Schüler Eltern – und auch Lehrer,,574,299824032
1000163602560555666,299972800,News,Strengere Regeln für Schüler Eltern – und auch Lehrer,,574,299809748
1000295479560700003,299836255,News,Blümel Kneissl &Co.: Das sind die Fixstarter,,574,299833840
1003281757411139142,299866366,News,"Trump sorgt mit ""Pocahontas""-Sager für Eklat",Moritz Gottsauner-Wolf,574,299831571
1004845194187735517,299866366,News,"Trump sorgt mit ""Pocahontas""-Sager für Eklat",Moritz Gottsauner-Wolf,574,299816215
1009895431438339410,293173038,,Heidi Strobl: Griessknödel mit karamellisierter Birne,Heidi Strobl,573,299816215
10102541958680734,299816215,News,Fahnenskandal von Mailand: Die Austria zeigt Flagge,A